In [1]:
import re
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import cv2
import glob
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.layers import Input, Dense, Flatten, BatchNormalization
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import SGD, Adam
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

2023-04-26 18:34:59.060611: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 18:35:02.749954: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sac0380/miniconda3/lib/python3.10/site-packages/cv2/../../lib64:
2023-04-26 18:35:02.750573: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sac0380/miniconda3/lib/python3.10/site-packages/cv2/.

In [2]:
data = pd.read_csv('TRY_final.txt', sep=',', encoding='latin-1')
data = data[data['mean_3110'].notna()]
data.head()

,AccSpeciesName,mean_14,mean_3117,mean_3106,mean_4,mean_3110,mean_26,stddev_14,stddev_3117,stddev_3106,stddev_4,stddev_3110,stddev_26,PlantGrowthForm
3,Abarema adenophora,24.422045,11.820481,NaN,0.663459,56861.990000,NaN,1.593152,1.030987,NaN,0.132959,NaN,NaN,tree
11,Abarema macradenia,23.394909,8.319237,31.670558,0.438332,16535.000000,90.000000,0.000000,0.129421,0.679612,0.000000,0.0,0.000000,tree
29,Abies alba,13.095739,7.846480,47.975610,0.499512,18.921429,70.544591,2.731756,2.738197,15.637276,0.076731,NaN,26.875742,tree
31,Abies balsamea,14.619919,16.226192,4.047670,0.457618,34.202755,7.473453,3.278028,6.272770,6.972198,0.093733,NaN,3.054439,tree
48,Abies fraseri,18.077778,5.566903,20.138400,NaN,22.801836,7.202712,3.111181,0.451749,5.774298,NaN,NaN,0.928773,tree


In [3]:
X = []
y_mean, y_std = [], []
for image in tqdm(glob.glob("images/*")):
    X.append(cv2.resize(cv2.imread(image), (224, 224)) / 255)
    y_mean.append(data.loc[data['AccSpeciesName'] == image[7:re.search(r"\d", image).start()], 'mean_3110'].values[0])

100%|██████████| 11270/11270 [00:31<00:00, 353.70it/s]


In [4]:
scaler = MinMaxScaler().fit(np.array(y_mean).reshape(1, -1))
y_mean = scaler.transform(np.array(y_mean).reshape(1, -1))

In [8]:
X = np.array(X)
y_mean = np.array(y_mean[0])

: 

: 

In [1]:
X.shape, y_mean.shape

NameError: name 'X' is not defined

In [3]:
base_model = ResNet50V2(input_shape=[224, 224] + [3,],include_top=False, weights='imagenet')
base_model.trainable = False

2023-04-26 18:19:19.973358: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 18:19:20.665109: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 18:19:20.665254: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-26 18:19:20.666098: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

94668760/94668760 [==============================] - 2s 0us/step


In [4]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
x = base_model(x, training=False)
x = Flatten()(x)
x = Dense(8, activation = 'relu')(x)
output = Dense(1)(x)
model = tf.keras.Model(inputs, output)

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 8)                 802824

In [6]:
model.compile(loss = 'mean_squared_error' , optimizer = 'adam' , metrics = ['mse', 'mae'])

In [11]:
model.fit(np.array(X), np.array(y_mean), epochs=2, batch_size=32, validation_split=0.2)

2023-04-26 18:30:11.636575: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 5428641792 exceeds 10% of free system memory.


ValueError: Data cardinality is ambiguous:
  x sizes: 9016
  y sizes: 1
Make sure all arrays contain the same number of samples.

In [ ]:
np.array(X).shape, np.array(y_mean).shape